In [5]:
#
# TODO: Extraer data desde base de datos
# TODO: Agregar data básica del modelo
# * Agregar URLs a publicaciones, imágenes
# TODO: Hacer segundo research en base a los modelos a comparar
# ? SOP: https://docs.google.com/document/d/1zp91sPv0uYKKByQpszNrFQEFBcFzxP0HXX5vgh1azVw/edit?usp=sharing

In [6]:
from google import genai
import os
import sys
sys.path.append('../')  # subir 1 nivel desde notebooks a src
from config.paths import SRC_DIR
import pandas as pd

In [7]:
pais = "Colombia"

### Generar los datos básicos del modelo

In [8]:
co_inventory = r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Data\historical_data\history\CO\BaseCOv2.csv"
mx_inventory = r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Data\historical_data\history\MX\BaseMXv5.csv"
cl_inventory = r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Data\historical_data\history\CL-MOTORCYCLES\df_cl-motorcycles_history.csv"

In [ ]:
if pais == "Colombia":
    base_data = pd.read_csv(co_inventory)
    country_code = "CO"
elif pais == "Mexico":
    base_data = pd.read_csv(mx_inventory)
    country_code = "MX"
elif pais == "Chile":
    base_data = pd.read_csv(cl_inventory)
    country_code = "CL"

C:\Users\JTRUJILLO\AppData\Local\Temp\ipykernel_24500\2436451131.py:2: DtypeWarning: Columns (8,23,30) have mixed types. Specify dtype option on import or set low_memory=False.
  base_data = pd.read_csv(co_inventory)


In [ ]:
# Filtrar por fecha de actualización
filtro_fecha = base_data["Fecha de actualización"] == "22/12/2025"
base_filtrada = base_data[filtro_fecha]

# Filtrar por estado disponible o sin stock
estados_validos = ["available", "no_stock"]
base_filtrada = base_filtrada[base_filtrada["Estado"].isin(estados_validos)]

# Excluir los que tienen redirección
base_filtrada = base_filtrada[base_filtrada["tiene_redirección"] == False]

# Seleccionar solo las columnas necesarias
columnas_a_usar = ["Fecha de actualización", "code", "Marca", "Modelo", "Año", "Precio Neto", "Tipo"]
df_columnas_seleccionadas = base_filtrada[columnas_a_usar].copy()
df_columnas_seleccionadas = df_columnas_seleccionadas.rename(columns={"Precio Neto": "Precio"})

# Eliminar duplicados por código
df_desduplicadas = df_columnas_seleccionadas.drop_duplicates(subset="code").copy()

# Agregar columna de URL
df_desduplicadas["url"] = df_desduplicadas["code"].apply(
    lambda x: f"https://www.galgo.com/{country_code.lower()}/motos/{x}"
)

# Definir columnas finales
columnas_finales = ["Fecha de actualización", "code", "Marca", "Modelo", "Año", "Tipo", "Precio", "url"]
df_columnas_finales = df_desduplicadas[columnas_finales]

In [11]:
df_modelo = df_columnas_finales[df_columnas_finales["code"] == "CO2961-hero-hunk-125-r"]

In [ ]:
for index, row in df_modelo.iterrows():
    marca = row["Marca"]
    modelo = row["Modelo"]
    año = row["Año"]
    pais = pais
    tipo = row["Tipo"]
    code = row["code"]

### Prompt a enviar

In [ ]:
from utils.prompt_utils import *

prompt_from_file = read_prompt_from_file(os.path.join(SRC_DIR, "data", "prompts", "knowledge_base_template.md"))
prompt = replace_variables(prompt_from_file, {
    "{marca}": marca,
    "{modelo}": modelo,
    "{año}": str(año),
    "{pais}": pais,
    "{tipo}": tipo
})

### Usando Gemini

In [ ]:
client = genai.Client() # La API Key se lee automaticamente desde variables de entorno

# Llamada a la API
interaction_con = client.interactions.create(
    model='gemini-3-pro-preview',
    input=prompt,
    tools=[{'type': 'google_search'}]  # Habilita Google Search
)

### Output de la API

In [ ]:
final_output = ""
for output in interaction_con.outputs:
    if hasattr(output, "text"):
        final_output += output.text

### Exportar OUTPUT

In [ ]:
with open(f"{SRC_DIR}/data/output_gemini/{code}-{pais}.md", "w", encoding="utf-8") as f:
    f.write(final_output)

### Usando Replicate

In [ ]:
# client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

# output = client.run(
#     ref = "google/gemini-3-pro",
#     input={
#         "prompt": prompt
#     },
# )

# print("\n--- RESPUESTA ---\n")
# if isinstance(output, list):
#     print("".join([str(x) for x in output]))
# else:
#     print(output)